# Reconstructing pathways cellulose degradation and updake

In [ ]:
import pandas as pd
from reframed import Metabolite, GPRAssociation, Gene, Protein, ReactionType, CBReaction, Environment, FVA, load_cbmodel
from reframed.io.sbml import parse_gpr_rule
import json

## Reconstructing pathway

### Metabolites
    

In [ ]:
metabolites = pd.read_excel('../input/Polysaccharide degrading pathways.xlsx', sheet_name="Mets.  Cat. of cellodextrins ", usecols="A:E").dropna()

In [ ]:
mets = []

for index, row in metabolites.iterrows():
    met = Metabolite(met_id=row['Identifier'], name=row['Name'], compartment="C_"+row['Compartment'])
    met.metadata={'FORMULA':row['Formula'], 'CHARGE':str(row['Charge'])}                  
    mets.append(met)

### Genes

In [ ]:
gene_id_map = pd.read_csv('../input/uniprot-proteome_UP000001349.tab', sep='\t', usecols=[3,4], names=['name', 'refseq'], header=0)
gene_id_map['name'] =  gene_id_map['name'].str.split()
gene_id_map = gene_id_map.explode('name').dropna()
gene_id_map['refseq'] = gene_id_map['refseq'].apply(lambda x: f'G_{x[:-3]}_1')
gene_id_map = dict(gene_id_map.values)

In [ ]:
def gene_str_to_GPR(gene_string):
    gpr = parse_gpr_rule(gene_string)
    
    for protein in gpr.proteins:
        protein.genes = [gene_id_map.get(x, 'G_' + x) for x in protein.genes]
                
    return gpr

### Reactions

In [ ]:
reactions = pd.read_excel('../input/Polysaccharide degrading pathways.xlsx', sheet_name="Rxns.  Cat. of cellodextrins ", usecols="A:F")

In [ ]:
rxns=[]
gprs={}

for index, row in reactions.iterrows():
    
    reaction_id = row['Identifier']
    name = row['Enzyme']
    reversible = False
    stoichiometry = json.loads(row['Stoichiometry'])
    reaction_type = ReactionType.ENZYMATIC
    
    if row['Transport']==1:
        reaction_type=ReactionType.TRANSPORT 
    
    rxn = CBReaction(reaction_id=reaction_id, name=name, reversible=reversible, stoichiometry=stoichiometry, reaction_type=reaction_type)
    rxns.append(rxn)
    
    gprs[reaction_id] = gene_str_to_GPR(row['Gene'])

### Exchange reactions

In [ ]:
mets_exchange = [met.id for met in mets if met.compartment=="C_e"]
rxns_exchange = []

for met in mets_exchange:
    rxn_id = "R_EX_" + met[2:]
    name = "Exchange of " + model.metabolites[met].name
    rxn = CBReaction(reaction_id=rxn_id, name=name, reversible=True, stoichiometry={met: -1.0} , reaction_type=ReactionType.EXCHANGE)
    rxns_exchange.append(rxn)

### Add new metabolites and reactions to model

In [ ]:
model = load_cbmodel('../models/RcH10_draft.xml')

In [ ]:
for met in mets:
    model.add_metabolite(met)
    
for rxn in rxns:
    model.add_reaction(rxn)
    model.set_gpr_association(rxn.id,gprs[rxn.id])
    
for rxn in rxns_exchange:
    model.add_reaction(rxn)